# Weak references

https://docs.python.org/3/library/weakref.html

In [1]:
import gc
import weakref

In [2]:
class C: pass

In [3]:
c = C()

In [4]:
repr(c)

'<__main__.C object at 0x000002427258F7F0>'

In [5]:
print(c)

In [6]:
r = weakref.ref(c)

In [7]:
r

<weakref at 0x00000242725C2250; to 'C' at 0x000002427258F7F0>

In [8]:
print(r())

In [9]:
r() is c

True

In [10]:
del c

In [11]:
print(r())

None


In [12]:
n = None

In [13]:
n

In [14]:
rn = weakref.ref(n)

TypeError: cannot create weak reference to 'NoneType' object

In [15]:
rone = weakref.ref(1)

TypeError: cannot create weak reference to 'int' object

In [16]:
class CSlots: 
    __slots__ = ()

In [17]:
weakref.ref(CSlots())

TypeError: cannot create weak reference to 'CSlots' object

In [18]:
class DSlots: 
    __slots__ = ('__dict__',)

In [19]:
weakref.ref(DSlots())

TypeError: cannot create weak reference to 'DSlots' object

In [20]:
DSlots().__dict__

{}

In [21]:
class WRSlots: 
    __slots__ = ('__weakref__',)

In [22]:
wr = WRSlots()

In [23]:
weakref.ref(wr)

<weakref at 0x00000242741298A0; to 'WRSlots' at 0x0000024273821F00>

In [24]:
weakref.ref(WRSlots())

<weakref at 0x00000242741290D0; dead>

In [25]:
c2 = C()

In [26]:
wr = weakref.ref(c2, lambda x: print(f"{x}'s referent is dead!"))

In [27]:
wr

<weakref at 0x0000024274129990; to 'C' at 0x00000242738607C0>

In [28]:
del c2

<weakref at 0x0000024274129990; dead>'s referent is dead!


In [29]:
wr

<weakref at 0x0000024274129990; dead>

In [30]:
wr2 = weakref.ref(C(), lambda: print("The referent is dead!"))

Exception ignored in: <function <lambda> at 0x0000024274159BD0>
Traceback (most recent call last):
  File "C:\Users\User\AppData\Local\Temp\ipykernel_4076\3704447829.py", line 1, in <module>
TypeError: <lambda>() takes 0 positional arguments but 1 was given


In [31]:
wr2

<weakref at 0x0000024274111300; dead>

In [32]:
cycle = C()

In [33]:
cycle.x = cycle

In [34]:
wrcycle = weakref.ref(cycle, lambda x: print("Referent (cycle) has died!"))

In [35]:
del cycle

In [36]:
gc.collect()

Referent (cycle) has died!


671

In [37]:
class Box: 
    
    def __init__(self, value): 
        self.value = value
    
    def __repr__(self): 
        return f'{type(self).__name__}({self.value!r})'
    
    def __eq__(self, other): 
        if not isinstance(other, type(self)): 
            return NotImplemented
        return self.value == other.value

In [38]:
x = Box(1)
y = Box(1)

In [39]:
x == y

True

In [40]:
print(x)

Box(1)


In [41]:
r1 = weakref.ref(x)
r2 = weakref.ref(x)

In [42]:
r1 == r2

True

In [43]:
r1 is r2  # Not guaranteed.

True

In [44]:
r3 = weakref.ref(x, lambda _: print("Dead box"))

In [45]:
r3 is r1

False

In [46]:
r3 == r1

True

In [47]:
s = weakref.ref(y)

In [48]:
r1 == s  # Weak references compare equal when their referents compare equal. 

True

In [49]:
hash(s)

TypeError: unhashable type: 'Box'

In [50]:
z = Box(1)

In [51]:
t = weakref.ref(z)

In [52]:
r1 == s == t

True

In [53]:
z.value = 2

In [54]:
print(z)

Box(2)


In [55]:
s == t

False

In [56]:
del x

Dead box


In [57]:
r1 == r2

True

In [58]:
r1 is r2

True

In [59]:
r1 == r3

False

In [60]:
print(y)

Box(1)


In [61]:
print(z)

Box(2)


In [62]:
s == t

False

In [63]:
z.value = 1

In [64]:
s == t

True

In [65]:
import math

In [66]:
nbox = Box(math.nan)

In [67]:
nbox == nbox

False

In [68]:
nbref = weakref.ref(nbox)

In [69]:
nbref == nbref

False

In [70]:
del nbox

In [71]:
nbref == nbref

True

In [72]:
class LockBox: 
    
    def __init__(self, value): 
        self._value = value
    
    def __repr__(self): 
        return f'{type(self).__name__}({self.value!r})'
    
    def __eq__(self, other): 
        if not isinstance(other, type(self)): 
            return NotImplemented
        return self.value == other.value

    def __hash__(self): 
        return hash(self.value)

    @property
    def value(self): 
        return self._value

In [73]:
lb = LockBox(1)

In [74]:
print(lb)

LockBox(1)


In [75]:
g = weakref.ref(lb)

In [76]:
hash(g)

1

In [77]:
del lb

In [78]:
g

<weakref at 0x00000242740A3600; dead>

In [79]:
hash(g)

1

In [80]:
lb2 = LockBox(2)

In [81]:
h = weakref.ref(lb2)

In [82]:
del lb2

In [83]:
hash(h)

TypeError: weak object has gone away